In [1]:
import numpy as np
import pyvista as pv
import potpourri3d as pp3d

In [7]:
mesh = pv.read("/Users/patrykrygiel/Downloads/hdd-cell-t008.vtp")
field = mesh["wss"]

dec = pp3d.DifferentialExteriorCalculus(mesh.points, mesh.regular_faces)
exact, coexact, harmonic = dec.decompose_field(field)

mesh["exact"] = exact
mesh["coexact"] = coexact
mesh["harmonic"] = harmonic
mesh["wss-rec"] = exact + coexact + harmonic
mesh["wss-mae"] = np.abs(mesh["wss"] - mesh["wss-rec"])

mesh.save("hhd-py.vtp")

print((field - (exact + coexact + harmonic)).mean())

-0.0011830106906200787


In [27]:
pl = pv.Plotter(shape=(2, 3), window_size=(1200, 800))

# Reconstruction
pl.subplot(0, 0)
pl.add_text("Original WSS")
pl.add_mesh(mesh.copy(), scalars="wss", clim=[0, 30])

pl.subplot(0, 1)
pl.add_text("Reconstructed WSS")
pl.add_mesh(mesh.copy(), scalars="wss-rec", clim=[0, 30])

pl.subplot(0, 2)
pl.add_text("Reconstruction MAE")
pl.add_mesh(mesh.copy(), scalars="wss-mae", clim=[0, 5])

# Components
for i, name in enumerate(["exact", "coexact", "harmonic"]):
    pl.subplot(1, i)
    pl.add_text(name)
    pl.add_mesh(mesh.copy(), color="white", opacity=0.3)
    pl.add_arrows(
        mesh.cell_centers().points, 
        mesh[name] / np.linalg.norm(mesh[name], axis=1, keepdims=True), 
        mag=0.02,
        show_scalar_bar=False
    )

pl.link_views()
pl.show()

Widget(value='<iframe src="http://localhost:57858/index.html?ui=P_0x39a0b2e50_18&reconnect=auto" class="pyvist…